In [50]:
"""

LINKS:

- https://github.com/sagemath/sage/issues/38456
- https://mathoverflow.net/questions/271932/formula-for-the-frobenius-schur-indicator-of-a-finite-group
- https://math.stackexchange.com/questions/832173/structure-of-g-invariant-bilinear-forms-over-finite-fields

""";

In [1]:
"""
NOTES:

For a unitary change-of-basis for a rep'n \rho of S_n over finite fields, we need:

1) find a S_n-invariant symmetric bilinear form w/ associated matrix U
2) factor U = AA^*, where * denotes conjugate-transpose w.r.t. x |--> x^q conjugation

1 sol'n) let U be unknown in variables over a polynomial ring over GF(q^2). use linear algebra to solve \rho(g)^T U \rho(g) == U
2 sol'n) use the GAP `forms` package BaseChangeToCanonical method to find the matrix factorization U = AA^* over F_{q^2}.

Then \tilde{\rho}(g) = A^*\rho(g)A^*.inverse() is unitary for all g \in G.

""";

In [5]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(GF(q**2),[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [1]:
def invariant_symmetric_sesquilinear_matrix(q,partition):
    """
    Computes the matrix of a S_n-invariant symmetric sesquilinear form.

    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 

    The equations are \rho(g)^T*U*\overline{\rho(g)} = \lambda_g*U where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}.

    The variables for U can be extracted to yield a matrix over GF(q^2) for each g.
    
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.

    Note: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T} to explicitly obtain the system.
    
    """

    # Define the group G and its rep'n as a Specht module, dimension
    n = sum(partition)
    SGA = SymmetricGroupAlgebra(GF(q^2), n)
    SM = SGA.specht_module(partition)
    G = SGA.group()
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    
    # Initialize U as a matrix of variables over GF(q^2)
    R = PolynomialRing(GF(q^2), 'u', d_rho^2)
    U_vars = R.gens()  # List of variable generators for U
    U = Matrix(R, d_rho, d_rho, U_vars)  # U is a d_rho x d_rho matrix of variables
    
    # for each generator of G, form the augmented system 
    def augmented_matrix(g):

        #compute \rho(g), transpose, conjugate
        rho_g = rho(Permutation(g))
        rho_g_T = rho_g.transpose()
        rho_g_conj = conjugate_pos_char(rho_g)
    
        # Compute lambda_g
        det_rho_g = det(rho_g)
        lambda_g = det_rho_g * (det_rho_g ** q)
    
        # Form the matrix equation \rho(g)^T*U*\overline{\rho(g)} = \lambda_g * U
        equation_matrix = rho_g_T*U*rho_g_conj - lambda_g * U
    
        # Initialize a list to hold rows of the augmented system
        augmented_system = []
    
        # Extract coefficients for each linear equation in the matrix
        for i in range(d_rho):
            for j in range(d_rho):
                # Get the (i, j) entry of the equation matrix, which is a linear combination of the u variables
                linear_expression = equation_matrix[i, j]
            
                # Extract the coefficients of each u_k in the linear expression
                row = [linear_expression.coefficient(u) for u in U_vars]
            
                # Append the row to the augmented system
                augmented_system.append(row)
    
        # Convert the augmented system to a matrix
        return Matrix(GF(q^2), augmented_system)

    #stack linear systems for each g in G
    total_system = matrix(GF(q^2),0,d_rho^2)
    for g in G:
        total_system = total_system.stack(augmented_matrix(g))
    
    #compute the null space of the overall matrix
    null_space = total_system.right_kernel()
    
    #return a d_rho x d_rho matrix over GF(q^2) from the 1 dim'l null space given as vector
    U_mats = [matrix(GF(q^2),d_rho,d_rho,b) for b in null_space.basis()]

    #verify that a solution to the linear system satisfies the G-invariance property
    assert all(rho(g).transpose()*U_mats[0]*conjugate_pos_char(rho(g)) == U_mats[0] for g in G)
    
    return U_mats

In [2]:
#define a general bi/sesquilinear form with the matrix U
def sesquilinear_form(x,y,U):
    return x.transpose()*U*y

In [3]:
#ensure the resulting form is G-invariant, symmetric, bi/sesquilinear by symbolic verification
def check_form_properties(q,partition):
    #define the representation matrix corresponding to q, partition
    SGA = SymmetricGroupAlgebra(GF(q^2),sum(partition))
    SM = SGA.specht_module(partition)
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    G = SGA.group()

    #define variables as polynomial generators
    R_xy = PolynomialRing(GF(q^2), d_rho, var_array='x,y')
    x = matrix([R_xy.gens()[2*i] for i in range(d_rho)]).transpose()
    y = matrix([R_xy.gens()[2*i+1] for i in range(d_rho)]).transpose()
    R_xy_lambda = PolynomialRing(R_xy,'lambda')
    lambda_ = R_xy_lambda.gens()[0]

    #compute the bilinear form matrix. coerce over polynomial ring
    U_mats = invariant_symmetric_sesquilinear_matrix(q,partition)
    U_mat = U_mats[0]
    if len(U_mats) > 1:
        print("Space of G-invariant symmetric bilinear forms has dimension > 1 for la=",partition)
        print("Dimension of space=",len(U_mats))
    U_form = matrix(R_xy_lambda,U_mat)
    
    #check symmetric property
    symmetric = sesquilinear_form(x,y,U_form) == sesquilinear_form(y,x,U_form)
    
    #check G-invariance property
    G_invariant = all(sesquilinear_form(rho(g)*x,rho(g)*y,U_form) == sesquilinear_form(x,y,U_form) for g in G)
    
    #check sesquilinear property. ISSUE: lambda_^q is a power of the ring generator, i.e. doesn't simplify.
    first_arg = sesquilinear_form(lambda_*x,y,U_form) == lambda_*sesquilinear_form(x,y,U_form)
    second_arg = sesquilinear_form(x,lambda_*y,U_form) == lambda_*sesquilinear_form(x,y,U_form) #need to amend for conjugation
    bilinear = first_arg and second_arg

    return symmetric and G_invariant and bilinear

In [6]:
#print the checks for pairs (p,la) where p is a prime and la is a partition of n
for p in Primes()[:3]:
    print(f"p={p}")
    for n in range(3,6):
        print([(la,check_form_properties(p,la)) for la in Partitions(n)])
    print("---------")

p=2
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
Space of G-invariant symmetric bilinear forms has dimension > 1 for la= [3, 1, 1]
Dimension of space= 2
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------
p=3
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------
p=5
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------


In [7]:
#NOTE: for q=2, la=[3,1,1] we get a two dim'l space
invariant_symmetric_sesquilinear_matrix(2,[3,1,1])

[
[1 1 1 1 1 0]  [0 0 0 0 0 1]
[1 1 1 1 0 1]  [0 0 0 0 1 0]
[1 1 1 0 1 1]  [0 0 0 1 0 0]
[1 1 0 1 1 1]  [0 0 1 0 0 0]
[1 0 1 1 1 1]  [0 1 0 0 0 0]
[0 1 1 1 1 1], [1 0 0 0 0 0]
]

In [8]:
#the G-invariant, symmetric, bilinear form is not the standard form. needs a different basis
U_mat = invariant_symmetric_sesquilinear_matrix(5,[3,1])[0]; U_mat

[1 3 3]
[3 1 3]
[3 3 1]

In [9]:
#to explain q=2, la=[3,1,1] note that the one of the composition factors (the irreducible quotients from any composition series)
#has multiplicity 2. we see this by looking at the Brauer character of the composition factor, and see (1,1,1) appears twice.
S5 = SymmetricGroup(5)
SGA = S5.algebra(GF(2))
for la in Partitions(5):
    M = SGA.specht_module(la)
    print(la, [V.brauer_character() for V in M.composition_factors()])

[5] [(1, 1, 1)]
[4, 1] [(4, 1, -1)]
[3, 2] [(4, -2, -1), (1, 1, 1)]
[3, 1, 1] [(1, 1, 1), (4, -2, -1), (1, 1, 1)]
[2, 2, 1] [(1, 1, 1), (4, -2, -1)]
[2, 1, 1, 1] [(4, 1, -1)]
[1, 1, 1, 1, 1] [(1, 1, 1)]


In [10]:
#print the matrix associated to the bilinear form for each (q,partition)
#note q needs to be large enough. p \nmid n! seems sufficient
for la in Partitions(5):
    U_mats = invariant_symmetric_sesquilinear_matrix(2,la)
    if len(U_mats) > 1:
        print("space of G-invariant symmetric bilinear forms has dimension greater than 1")
    print("la =",la)
    print("q =",3)
    print(U_mats[0])
    print("--------")

la = [5]
q = 3
[1]
--------
la = [4, 1]
q = 3
[0 1 1 1]
[1 0 1 1]
[1 1 0 1]
[1 1 1 0]
--------
la = [3, 2]
q = 3
[0 0 0 1 1]
[0 0 1 0 1]
[0 1 0 0 1]
[1 0 0 0 0]
[1 1 1 0 0]
--------
space of G-invariant symmetric bilinear forms has dimension greater than 1
la = [3, 1, 1]
q = 3
[1 1 1 1 1 0]
[1 1 1 1 0 1]
[1 1 1 0 1 1]
[1 1 0 1 1 1]
[1 0 1 1 1 1]
[0 1 1 1 1 1]
--------
la = [2, 2, 1]
q = 3
[1 1 1 1 1]
[1 1 1 1 1]
[1 1 1 1 1]
[1 1 1 1 1]
[1 1 1 1 1]
--------
la = [2, 1, 1, 1]
q = 3
[0 1 1 1]
[1 0 1 1]
[1 1 0 1]
[1 1 1 0]
--------
la = [1, 1, 1, 1, 1]
q = 3
[1]
--------


In [11]:
#define the representation matrix corresponding to q, partition
#define generators of multiplicative groups of GF(q) and GF(q^2), w and z respectively
q = 5; la = [2,1,1]
SGA = SymmetricGroupAlgebra(GF(q^2),sum(la))
SM = SGA.specht_module(la)
rho = SM.representation_matrix
d_rho = SM.dimension()
G = SGA.group()
F = GF(q^2)
z = F.gen()
w = F.subfield(1).multiplicative_generator()

In [12]:
#compute the S_n-invariant symmetric bilinear form associated to (q,la)
U = invariant_symmetric_sesquilinear_matrix(q,la)[0]; U

[1 2 3]
[2 1 2]
[3 2 1]

In [27]:
libgap.LoadPackage("forms")
u_gap=libgap([list(row) for row in U]); u_gap

[ [ Z(5)^0, Z(5), Z(5)^3 ], [ Z(5), Z(5)^0, Z(5) ], [ Z(5)^3, Z(5), Z(5)^0 ] ]

In [38]:
#use libgap.eval for GAP evalutation of BaseChangeToCanonical using `forms` package
def base_change_matrix(U,q):
    libgap.LoadPackage("forms")
    u_gap = libgap([list(row) for row in U])
    libgap.eval("u := " + str(u_gap))
    libgap.eval(f"form := HermitianFormByMatrix(u, GF({q}^2));")
    b_str = libgap.eval("b := BaseChangeToCanonical(form);")
    return matrix(F,libgap(b_str)).inverse()

In [39]:
#compute the factorization U = AA^*
A = base_change_matrix(U,q)
A_star = conjugate_pos_char(A).transpose()
U == A*A_star

True

In [15]:
#construct the unitary representations as \tilde{\rho}(g) = A^*\rho(g)A^*.inverse()
all_unitary = True
for g in G:
    tilde_rho_g = A_star*rho(g)*A_star.inverse()
    unitary_check = conjugate_pos_char(tilde_rho_g).transpose()*tilde_rho_g == identity_matrix(F,d_rho)
    all_unitary = all_unitary and unitary_check
print(all_unitary)

True
